In [23]:
import os.path
import pdfkit
import sys
import PyPDF2
import pandas as pd
import PythonMagick
import io
import ast
import json
from google.cloud.vision import types
from google.cloud import vision
from os.path import isfile, join
sys.path.append('../')
from modules.helpers.dataframe_to_json import DataframeToJSON

In [24]:
def detect_text(path):
    df = pd.DataFrame()
    page = path.split('.')[1].replace('pdf','')
    
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    row_dict = {'WordText':None, 'PageNumber': None, 'X_Min': None, 'X_Max': None, 'Y_Min': None, 'Y_Max': None}
    for text in texts:

        row_dict['WordText'] = text.description

        vertices = ([[vertex.x, vertex.y]
                    for vertex in text.bounding_poly.vertices])
        
        xlist = []
        ylist = []
        for vertex in vertices:
            for v in vertex:
                xlist.append(vertex[0])
                ylist.append(vertex[1])
        row_dict['X_Min'] = min(xlist)
        row_dict['X_Max'] = max(xlist)
        row_dict['Y_Min'] = min(ylist)
        row_dict['Y_Max'] = max(ylist)
        
        row_dict['PageNumber'] = page

        df = df.append(row_dict, ignore_index=True)
    df = df[pd.notnull(df['WordText'])]
    return ast.literal_eval(DataframeToJSON(df))

## HTML to PDF

In [31]:
mypath = 'C:/Users/juang/Documents/Stocks/'
tickers = []
config = pdfkit.configuration(wkhtmltopdf='C:\Program Files\wkhtmltopdf\\bin\\wkhtmltopdf.exe')
files = [f for f in os.listdir(mypath) if '.html' in f]
for f in files:
    tickers.append(f.split('-')[0])
    if '.html' in f and not os.path.isfile(mypath+'PDFs/'+f.split('-')[0]+'.pdf'): 
        pdf = f.split('-')[0]+'.pdf'
        pdfkit.from_file(mypath+f, mypath+'PDFs/'+pdf, configuration=config)

In [32]:
tickers

['CHK', 'DIS', 'F', 'FB', 'GME', 'LLY', 'RAD', 'UPS']

## PDF to Images

In [34]:
mypath = 'C:/Users/juang/Documents/Stocks/PDFs/'
onlyfiles = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]
for f in onlyfiles:
    directory = mypath+str(f).replace('.pdf','')
    if not os.path.exists(directory):
        os.makedirs(directory)
        pdffilename = f 
        pdf_im = PyPDF2.PdfFileReader(open(mypath+pdffilename, "rb"))
        npage = pdf_im.getNumPages()
        print('Converting %d pages.' % npage)
        for p in range(npage):
            im = PythonMagick.Image()
            im.density('300')
            im.read(mypath+pdffilename + '[' + str(p) +']')
            im.write(directory+'/'+pdffilename + str(p)+ '.jpg')
    elif not len(os.listdir(directory)) >= 1:
        pdffilename = f 
        pdf_im = PyPDF2.PdfFileReader(open(mypath+pdffilename, "rb"))
        npage = pdf_im.getNumPages()
        print('Converting %d pages.' % npage)
        for p in range(npage):
            im = PythonMagick.Image()
            im.density('300')
            im.read(mypath+pdffilename + '[' + str(p) +']')
            im.write(directory+'/'+pdffilename + str(p)+ '.jpg')

Converting 46 pages.


## Images to JSON

In [35]:
mypath = 'C:/Users/juang/Documents/Stocks/PDFs/'
for i in range(0, len(tickers)):
    json_dict = {'Ticker': None, 'Pages': []}
    json_dict['Ticker'] = tickers[i]
    onlyfiles = [f for f in os.listdir(mypath+tickers[i].replace('.pdf',''))]
    tickers[i] = tickers[i].replace('.pdf','')
    print(tickers[i])
    
    if os.path.isfile(mypath+"/"+tickers[i]+"/"+'all'+'.json'):
        if os.path.getsize(mypath+"/"+tickers[i]+"/"+'all'+'.json') < 100:
            for f in onlyfiles:
                if '.pdf' in f and os.path.getsize(mypath+"/"+tickers[i]+"/"+'all'+'.json') < 100:
                    json_row = detect_text(mypath+tickers[i]+'/'+f)
                    json_dict['Pages'].append(json_row)
    
                    with open(mypath+"/"+tickers[i]+"/"+'all'+'.json', 'w') as fp:
                        json.dump(json_dict, fp)
    else:
        for f in onlyfiles:
            json_row = detect_text(mypath+tickers[i]+'/'+f)
            json_dict['Pages'].append(json_row)

            with open(mypath+"/"+tickers[i]+"/"+'all'+'.json', 'w') as fp:
                json.dump(json_dict, fp)

CHK
DIS
F
FB
GME
LLY
RAD
UPS
